In [ ]:
import os
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from tqdm import *
from scipy import stats

from pathlib import Path

NathanPath="d:\Documents\Info\INF554\INF554_Kaggle_Project"
NathanPath="/users/eleves-a/2019/nathan.peluso/INF554/INF554_Kaggle_Project"

In [ ]:
project_path = str(Path(os.getcwd()).parent.absolute())
print("Current directory : " + os.getcwd() + ", Project directory : " + project_path)

In [ ]:
os.chdir(project_path)
os.chdir(NathanPath)

In [ ]:

# Read training data
df_train = pd.read_csv('data/train.csv', dtype={'author': np.int64, 'hindex': np.float32})
n_train = df_train.shape[0]

# Read test data
df_test = pd.read_csv('data/test.csv', dtype={'author': np.int64})
n_test = df_test.shape[0]

with open("data/abstracts.txt", encoding="utf8") as file:
    texts = file.readlines()

## I) Create indicators (do not rerun)

In [ ]:
# Load the graph
G = nx.read_edgelist('data/coauthorship.edgelist', delimiter=' ', nodetype=int)
n_nodes = G.number_of_nodes()
n_edges = G.number_of_edges() 
print('Number of nodes:', n_nodes)
print('Number of edges:', n_edges)

In [ ]:
nx.set_node_attributes(G,dict(nx.degree(G)),"degree")
nx.set_node_attributes(G,dict(nx.clustering(G)),"clustering_coef")
nx.set_node_attributes(G,dict(nx.average_neighbor_degree(G)),"mean_neighbors_degree")

In [ ]:
nx.readwrite.graphml.write_graphml(G,'Indicators/graph_with_ind2.xml')

## II) Use indicators

In [ ]:
G2=nx.readwrite.graphml.read_graphml('Indicators/graph_with_ind2.xml')

In [10]:
train_dict=[(a,b) for a,b in np.array(df_train, dtype=np.int64)]
id_train=[str(i) for i,_ in train_dict]

G_train=G2.subgraph(id_train)
print(G_train)

h_index_train=[x for _,x in train_dict]
degree_train=np.array([d for x,d in G_train.nodes("degree")])
clust_coef_train=np.array([d for x,d in G_train.nodes("clustering_coef")])
#sq_clust_coef_train=np.array([d for x,d in G_train.nodes("sq_clustering_coef")])
neighb_degree_train=np.array([d for x,d in G_train.nodes("mean_neighbors_degree")])


def normalize(X):
    return (X-np.mean(X))/np.std(X)
n_h_index_train=normalize(h_index_train)
n_degree_train=normalize(degree_train)
n_clust_coef_train=normalize(clust_coef_train)
#n_sq_clust_coef_train=normalize(sq_clust_coef_train)
n_neighb_degree_train=normalize(neighb_degree_train)

KeyboardInterrupt: 

In [12]:
core_number=np.array([nx.algorithms.core_number(G_train)[x] for x in G_train.nodes])

In [ ]:
n_core_number=normalize(core_number)

In [ ]:
nx.draw_random(G_train, node_size=10)
plt.title("Raw graph of train set")
plt.show()

In [ ]:
def cov(X,Y):
    return np.mean((X-np.mean(X))*(Y-np.mean(Y)))
def corr(X,Y):
    return np.sqrt(cov(X,Y))
def softmax(X):
    return np.exp(X)/(1+np.exp(X))

In [ ]:
chi2_h_degree=corr(n_h_index_train,n_degree_train)
chi2_h_clust_coef=corr(n_h_index_train,n_clust_coef_train)
#chi2_h_sq_clust_coef=corr(n_h_index_train,n_sq_clust_coef_train)
chi2_h_neighb_degree=corr(n_h_index_train,n_neighb_degree_train)
chi2_degree_neighb_degree=corr(n_degree_train,n_neighb_degree_train)
print(chi2_h_degree,chi2_h_clust_coef, chi2_h_neighb_degree, chi2_degree_neighb_degree)

Le h-index est peu corrélé de manière directe au degré et au degré des voisins (meileur h-index, id=2426470670, h-index=187, degré=8 et degré_voisins=11.75)

In [ ]:
colors=[[softmax(n_h_index_train[x]),(1-softmax(n_h_index_train[x])),0] for x in range(len(n_h_index_train))]
colors2=[ colors[x] if n_h_index_train[x]>np.mean(n_h_index_train) else [1,1,1] for x in range(len(n_h_index_train))]
plt.figure(figsize=(10, 10), dpi=80)
plt.scatter(n_neighb_degree_train,clust_coef_train, c=colors, s=1)

plt.show()

## III) Create author - text dict

In [ ]:
with open("data/author_papers.txt", 'r') as f:
    data=f.readlines()

In [ ]:
data2=[[x.split(':')[0], x.split(':')[1].split('\n')[0].split('-')] for x in data]
dict_papers=dict(data2)

## IV) With text

In [ ]:
import json

In [ ]:
with open('Exploration/Vectors_and_clusters.json', 'r') as f:
    data=json.load(f)

In [ ]:
data['3603']

In [ ]:
for author in dict_papers.keys():
    print(author + ":")
    for paper in dict_papers.get(author):
        print(paper)
    print("\n")